In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-11-22 20:16:20.940654: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-11-22 20:16:22 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-11-22 20:16:22 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-11-22 20:16:22 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-11-22 20:16:22 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = VAE(input_dim=(x_train.shape[0], x_train.shape[1]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes)

2023-11-22 20:16:22 [INFO]: Classificator available for the latent space of the autoencoder


In [5]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=2)

trained_model = pipe(train_data=train_data)

2023-11-22 20:16:22 [INFO]: +++ training_pipeline +++
2023-11-22 20:16:22 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-11-22_20-16-22


Epoch 1/2
[<tf.Tensor 'gradient_tape/encoder_1/z_mean_1/MatMul_1:0' shape=(16, 10) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/z_mean_1/add/Reshape:0' shape=(10,) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/z_log_var_1/MatMul_1:0' shape=(16, 10) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/z_log_var_1/add/Reshape:0' shape=(10,) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/conv2d_1/convolution/Conv2DBackpropFilter:0' shape=(3, 3, 1, 32) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/conv2d_1/Reshape:0' shape=(32,) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/conv2d_1_2/convolution/Conv2DBackpropFilter:0' shape=(3, 3, 32, 64) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/conv2d_1_2/Reshape:0' shape=(64,) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/dense_1/MatMul_1:0' shape=(3136, 16) dtype=float32>, <tf.Tensor 'gradient_tape/encoder_1/dense_1/add/Reshape:0' shape=(16,) dtype=float32>, <tf.Tensor 'gradient_tape/decoder_1/dense_1_1/Ma

2023-11-22 20:16:35.203633: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc84c016d20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-22 20:16:35.203656: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-22 20:16:35.272254: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1700680596.194943   11205 device_compiler.h:187] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-22 20:16:36.196696: E external/local_xla/xla/stream_executor/stream_executor_internal.h:181] SetPriority unimplemented for this stream.



Epoch 1: total_loss improved from inf to 84909208.00000, saving model to ../output_dir/training_pipeline_2023-11-22_20-16-22/model.weights.h5
469/469 - 49s - 104ms/step - clf_loss: 24.7585 - kl_loss: 84901288.0000 - reconstruction_loss: 7217.4004 - total_loss: 84909208.0000
Epoch 2/2


KeyboardInterrupt: 

In [ ]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)

In [ ]:
import tensorflow as tf
tensor = tf.range(10)
tf.print(tensor, output_stream=sys.stderr)
